In [1]:
%useLatestDescriptors
%use lets-plot
%use dataframe

# Load data

In [2]:
import org.um.feri.ears.algorithms.gp.GPAlgorithmExecutor
import org.um.feri.ears.util.gp_stats.GPAlgorithmMultiConfigurationsProgressData
import org.um.feri.ears.util.gp_stats.GPProgramSolutionSimple

var dataFile = "C:\\Users\\marko\\UnityProjects\\GenIATraP_refactor\\GeneralTrainingEnvironmentForMAS\\evo_monitor\\src\\data\\multiConfigurationPrograssData.ser"
var progressData = GPAlgorithmMultiConfigurationsProgressData.deserializeState(dataFile)
System.out.println("Progress data loaded from file: " + progressData)

System.out.println("Data size: " +progressData.multiConfigurationProgressData.size)

Progress data loaded from file: org.um.feri.ears.util.gp_stats.GPAlgorithmMultiConfigurationsProgressData@5c034db4
Data size: 1


# Prepare helper classes & functions

In [3]:
class SelectedIndividual(
    var selectedConfiguration: Int,
    var selectedRun: Int,
    var selectedGeneration: Int,
    var bestWorstIndividual: Int, // 0 = best, 1 = worst
) {
    constructor() : this(0, 0, 0, 0)
}

In [4]:
// Prepare data (best, worst and average individual rating per generation)
fun prepareData(selectedConfiguration: Int, selectedRun: Int): Triple<MutableList<GPProgramSolutionSimple>, MutableList<GPProgramSolutionSimple>, MutableList<Double>> {
    val worstGenIndividuals =
        mutableListOf<GPProgramSolutionSimple>() // TODO - Add click option (or just pass as params?)
    val avgGenIndividuals = mutableListOf<Double>()
    val bestGenIndividuals =
        mutableListOf<GPProgramSolutionSimple>()  // TODO - Add click option (or just pass as params?)

    var multiConfigurationProgressData = progressData.multiConfigurationProgressData[selectedConfiguration]
    var multiRunProgressData = multiConfigurationProgressData.multiRunProgressData[selectedRun]

    for (genProgressData in multiRunProgressData.gensProgressData) {
        var bestGenIndividual = genProgressData.population[0]
        var worstIndividual = genProgressData.population[0]
        var avgGenIndividual = 0.0

        for (solution in genProgressData.population) {
            if (solution.finalIndividualFitness.additionalValues["Rating"]!! < bestGenIndividual.finalIndividualFitness.additionalValues["Rating"]!!) {
                bestGenIndividual = solution
            }

            if (solution.finalIndividualFitness.additionalValues["Rating"]!! > worstIndividual.finalIndividualFitness.additionalValues["Rating"]!!) {
                worstIndividual = solution
            }

            avgGenIndividual += solution.finalIndividualFitness.additionalValues["Rating"]!!
        }

        bestGenIndividuals.add(bestGenIndividual)
        worstGenIndividuals.add(worstIndividual)

        avgGenIndividual /= genProgressData.population.size
        avgGenIndividuals.add(avgGenIndividual)
    }

    return Triple(bestGenIndividuals, worstGenIndividuals, avgGenIndividuals)
}

In [7]:
import org.um.feri.ears.individual.representations.gp.IndividualMatchResult

fun sumIndividualValues(individualMatchResult: IndividualMatchResult): Double {
    var sum = 0.0
    for (value in individualMatchResult.individualValues.values) {
        sum += value
    }

    return sum
}

# Visualize data (line plot) - Best, worst and average individual rating per generation

In [15]:
// Input params for visualization START
val selectedConfiguration = 0
val selectedRun = 0
// Input params for visualization END

// Prepare data
val (bestGenIndividuals, worstGenIndividuals, avgGenIndividuals) = prepareData(selectedConfiguration, selectedRun)

val generations = mutableListOf<Int>()

for (i in 0 until bestGenIndividuals.size) {
    generations.add(i)
}

// Display data
val dfRating = dataFrameOf(
    "Generation" to generations + generations + generations,
    "Rating" to bestGenIndividuals.map { it.finalIndividualFitness.additionalValues["Rating"]!! } + worstGenIndividuals.map { it.finalIndividualFitness.additionalValues["Rating"]!! } + avgGenIndividuals,
    "Individual" to List(generations.size) { "Best" } + List(generations.size) { "Worst" } + List(generations.size) { "Average" }
)

val dfAvgFitness = dataFrameOf(
    "Generation" to generations + generations + generations,
    "Rating" to bestGenIndividuals.map { sumIndividualValues(it.finalIndividualFitness.avgMatchResult)  } + worstGenIndividuals.map { sumIndividualValues(it.finalIndividualFitness.avgMatchResult) } + avgGenIndividuals,
    "Individual" to List(generations.size) { "Best" } + List(generations.size) { "Worst" } + List(generations.size) { "Average" }
)

// The plot variable stores a plot with the temperature values for each city
val linePlotRating =
    letsPlot(dfRating.toMap()) { x = "Generation"; y = "Rating"; color = "Individual"} + ggsize(600, 500) + geomPoint(shape = 15) + geomLine() + ggtitle("Best Generation Rating")

val linePlotAvgFitness =
    letsPlot(dfAvgFitness.toMap()) { x = "Generation"; y = "Rating"; color = "Individual"} + ggsize(600, 500) + geomPoint(shape = 15) + geomLine() + ggtitle("Best Generation Average fitness")

val plots = listOf(linePlotRating, null, linePlotAvgFitness, null)
gggrid(plots, ncol = 2)


<path d="M12.60287254639406 154.28333822816282 L12.60287254639406 154.28333822816282 L40.6092559828253 170.16626618203605 L68.61563941925655 186.2965980884406 L96.62202285568779 183.57723484098838 L124.62840629211905 196.03356220375278 L152.6347897285503 196.42532184858862 L180.64117316498152 195.27663502180658 L208.6475566014128 198.0012757010425 L236.65394003784402 209.0454545454545 L264.6603234742753 209.0454545454545 " fill="none" stroke-width="1.6500000000000001" stroke="rgb(228,26,28)" stroke-opacity="1.0">
 
 
 
 
 
 
 
 <path d="M12.60287254639406 92.1436890512721 L12.60287254639406 92.1436890512721 L40.6092559828253 96.39223606483219 L68.61563941925655 102.92476114982927 L96.62202285568779 103.40212534954475 L124.62840629211905 113.76570955476248 L152.6347897285503 106.21981445242832 L180.64117316498152 121.79130975646865 L208.6475566014128 127.85537310556003 L236.65394003784402 129.42448346115768 L264.6603234742753 163.99277113103668 " fill="none" stroke-width="1.6500000000000001" stroke="rgb(77,175,74)" stroke-opacity="1.0">
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 2 
 
 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 6 
 
 
 
 
 
 
 
 
 8 
 
 
 
 
 
 
 
 
 
 
 -50 
 
 
 
 
 
 
 -40 
 
 
 
 
 
 
 -30 
 
 
 
 
 
 
 -20 
 
 
 
 
 
 
 -10 
 
 
 
 
 
 
 
 
 Best Generation Rating 
 
 
 
 
 Rating 
 
 
 
 
 Generation 
 
 
 
 
 
 
 
 
 Individual 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Best 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Worst 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Average 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M12.240117752963688 45.42441353430287 L12.240117752963688 45.42441353430287 L39.440379426216325 58.352916621354694 L66.64064109946897 27.724364555429414 L93.84090277272159 24.888143519903867 L121.04116444597423 23.662973312118467 L148.24142611922687 25.617444204120208 L175.4416877924795 209.04545454545456 L202.64194946573215 135.04570633929 L229.8422111389848 61.835195417278754 L257.0424728122374 61.835195417278754 " fill="none" stroke-width="1.6500000000000001" stroke="rgb(228,26,28)" stroke-opacity="1.0">
 
 
 
 <path d="M12.240117752963688 9.954545454545462 L12.240117752963688 9.954545454545462 L39.440379426216325 9.954545454545462 L66.64064109946897 10.186120162304366 L93.84090277272159 9.954545454545462 L121.04116444597423 9.954545454545462 L148.24142611922687 9.954545454545462 L175.4416877924795 9.954545454545462 L202.64194946573215 10.186120162304366 L229.8422111389848 35.94054429605994 L257.0424728122374 83.81407468876131 " fill="none" stroke-width="1.6500000000000001" stroke="rgb(55,126,184)" stroke-opacity="1.0">
 
 
 
 <path d="M12.240117752963688 25.30913989001008 L12.240117752963688 25.30913989001008 L39.440379426216325 25.885084368525483 L66.64064109946897 26.770651067318994 L93.84090277272159 26.835363844225913 L121.04116444597423 28.2402792380238 L148.24142611922687 27.21733736971574 L175.4416877924795 29.328251221902477 L202.64194946573215 30.15031198576589 L229.8422111389848 30.36302481266963 L257.0424728122374 35.049195014306584 " fill="none" stroke-width="1.6500000000000001" stroke="rgb(77,175,74)" stroke-opacity="1.0">
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 2 
 
 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 6 
 
 
 
 
 
 
 
 
 8 
 
 
 
 
 
 
 
 
 
 
 -300 
 
 
 
 
 
 
 -200 
 
 
 
 
 
 
 -100 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 Best Generation Average fitness 
 
 
 
 
 Rating 
 
 
 
 
 Generation 
 
 
 
 
 
 
 
 
 Individual 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Best 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Worst 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Average

# Visualize data (radar plot) - Comparison of individuals from different generations, runs & configurations

In [ ]:
// Input params for visualization START
val selectedIndividuals = mutableListOf<SelectedIndividual>()
selectedIndividuals.add(SelectedIndividual(0, 0, 3, 0)) // Individual at position
selectedIndividuals.add(SelectedIndividual(0, 0, 5, 0))
selectedIndividuals.add(SelectedIndividual(0, 0, 9, 0))
// Input params for visualization END

// Prepare data for visualization
var selectedGPAlgorithmSolutionsSimple = mutableListOf<GPProgramSolutionSimple>()

// Get selected individuals
for (selectedIndividual in selectedIndividuals){
    // Prepare data
    val (bestGenIndividuals, worstGenIndividuals, _) = prepareData(selectedIndividual.selectedConfiguration, selectedIndividual.selectedRun)

    var individual: GPProgramSolutionSimple?;
    if(selectedIndividual.bestWorstIndividual == 0) {
        individual = bestGenIndividuals[selectedIndividual.selectedGeneration]
    }
    else {
        individual = worstGenIndividuals[selectedIndividual.selectedGeneration]
    }

    selectedGPAlgorithmSolutionsSimple.add(individual!!)
}

val fitnessKeys = mutableListOf<String>()
for (individual in selectedGPAlgorithmSolutionsSimple) {
    for (key in individual.finalIndividualFitness.avgMatchResult.individualValues.keys) {
        if (key !in fitnessKeys) {
            fitnessKeys.add(key)
        }
    }
}

// Order fitnessKeys alphabetically
fitnessKeys.sort()

// Build labels for visualization
val labels = mutableListOf<String>()
for (individual in selectedGPAlgorithmSolutionsSimple) {
    labels.addAll(fitnessKeys)
}

val datasets = mutableListOf<MutableList<Double>>()

// Build datasets for visualization
for (individual in selectedGPAlgorithmSolutionsSimple) {
    val dataset = mutableListOf<Double>()
    for (key in fitnessKeys) {
        if(individual.finalIndividualFitness.avgMatchResult.individualValues[key] == null) {
            dataset.add(0.0)
            continue
        }
        dataset.add(-individual.finalIndividualFitness.avgMatchResult.individualValues[key]!!)
    }
    datasets.add(dataset)
}

// TODO Check if correct individuals are selected for visualization !!
// TODO Make this more generic (e.g., for more than 2 individuals)
var fitnessKeysIndexed = mutableListOf<Int>()
for (i in 0 until fitnessKeys.size) {
    fitnessKeysIndexed.add(i + 1)
}

var individualValueIdsMapped = mutableListOf<Int>()
var individualMapped = mutableListOf<String>()
var index = 0
for(dataset in datasets) {
    individualValueIdsMapped.addAll(fitnessKeysIndexed)
    individualMapped.addAll(List(fitnessKeys.size) { "Individual " + (index + 1) })
    index++
}

// Build map for visualization
val selectedIndividualsData = mapOf(
    "IndividualValueLabels" to labels,
    "IndividualValueId" to individualValueIdsMapped,
    "Individual" to individualMapped,
    "IndividualValues" to datasets.map { it }.toMutableList().flatten()
)

// Map each fitnessKey to a map 1 -> "fitnessKey1", 2 -> "fitnessKey2", ...
var labelsIndexed = fitnessKeys.mapIndexed { index, s -> index to s }.toMap().mapKeys { it.key + 1 }

letsPlot(selectedIndividualsData) +
        geomArea(flat = true) {   // <-- flat. I.e., do not transform segments to curves
            x = "IndividualValueId"; y = "IndividualValues"
        } +
        geomPoint() { x = "IndividualValueId"; y = "IndividualValues"; color="Individual" } +
        scaleXDiscrete(labels = labelsIndexed) +
        coordPolar()

# Individual node counts

In [ ]:
import org.jetbrains.letsPlot.intern.Plot

// Input params for visualization START
val selectedIndividuals = mutableListOf<SelectedIndividual>()
selectedIndividuals.add(SelectedIndividual(0, 0, 0, 0)) // Individual at position
selectedIndividuals.add(SelectedIndividual(0, 0, 3, 0))
selectedIndividuals.add(SelectedIndividual(0, 0, 9, 0))
// Input params for visualization END

// Prepare data for visualization
var selectedGPAlgorithmSolutionsSimple = mutableListOf<GPProgramSolutionSimple>()

// Get selected individuals
for (selectedIndividual in selectedIndividuals){
    // Prepare data
    val (bestGenIndividuals, worstGenIndividuals, _) = prepareData(selectedIndividual.selectedConfiguration, selectedIndividual.selectedRun)

    var individual: GPProgramSolutionSimple?;
    if(selectedIndividual.bestWorstIndividual == 0) {
        individual = bestGenIndividuals[selectedIndividual.selectedGeneration]
    }
    else {
        individual = worstGenIndividuals[selectedIndividual.selectedGeneration]
    }

    selectedGPAlgorithmSolutionsSimple.add(individual!!)
}

var plots = mutableListOf<Plot>()

for (individual in selectedGPAlgorithmSolutionsSimple) {
    var nodeCountsData = mapOf(
        "NodeName" to individual.nodeCounts.keys.toList(),
        "NodeCount" to individual.nodeCounts.values.toList()
    )

    val blankTheme = theme(line=elementBlank(), axis=elementBlank())
    val (w, h) = 400 to 400
    val p = letsPlot(nodeCountsData) + ggsize(w,h) + blankTheme

    val plot = p + geomPie(stat = Stat.identity,
            size = 20, stroke = 1, color = "white", hole = 0.5) { slice = "NodeCount"; fill = "NodeName" } +
            blankTheme +
            scaleFillBrewer(palette = "Set1") + ggtitle("Individual: " + individual.individualId)

    plots.add(plot)
}

gggrid(plots)